Gloabl Experiment Setup

In [1]:
import mlflow
# Set the MLflow tracking URI to a new SQLite URI
mlflow.set_tracking_uri("sqlite:///new_mlflow.db")
mlflow.set_experiment("LightGBM")

2024/04/25 15:58:03 INFO mlflow.tracking.fluent: Experiment with name 'LightGBM' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/arham/Downloads/Projects/03-Experiments/mlruns/4', creation_time=1714075083201, experiment_id='4', last_update_time=1714075083201, lifecycle_stage='active', name='LightGBM', tags={}>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
import lightgbm as lgb
from sklearn.metrics import accuracy_score

def load_data(path):
    df = pd.read_csv(path)
    train_df, test_df = train_test_split(df, test_size=0.35, random_state=42)
    train_df, val_df,  = train_test_split(train_df, test_size=0.20, random_state=42)
    train_df = train_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    test_df = test_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    val_df = val_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    return train_df, val_df, test_df

def encode_target(train):
    target_key = {'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Overweight_Level_I': 2, 'Overweight_Level_II': 3, 'Obesity_Type_I': 4,'Obesity_Type_II' : 5, 'Obesity_Type_III': 6}
    train['NObeyesdad'] = train['NObeyesdad'].map(target_key)
    return train

def make_gender_binary(train):
    train['Gender'] = train['Gender'].map({'Male':0, 'Female':1})

def datatypes(train):
    train['Weight'] = train['Weight'].astype(float)
    train['Age'] = train['Age'].astype(float)
    train['Height'] = train['Height'].astype(float)
    return train

# def age_binning(train_df):
#     train_df['Age_Group'] = pd.cut(train_df['Age'], bins=[0, 20, 30, 40, 50, train_df['Age'].max()], labels=['0-20', '21-30', '31-40', '41-50', '50+'])
#     return train_df

def age_binning(df):
    age_groups = []
    for age in df['Age']:
        if age <= 20:
            age_group = 1
        elif age <= 30:
            age_group = 2
        elif age <= 40:
            age_group = 3
        elif age <= 50:
            age_group = 4
        else:
            age_group = 5
        age_groups.append(age_group)
    df['Age_Group'] = age_groups
    return df

def age_scaling_log(train_df):
    train_df['Age'] = train_df['Age'].astype(float)
    train_df['Log_Age'] = np.log1p(train_df['Age'])
    return train_df

def age_scaling_minmax(train_df):
    train_df['Age'] = train_df['Age'].astype(float)
    scaler_age = MinMaxScaler()
    train_df['Scaled_Age'] = scaler_age.fit_transform(train_df['Age'].values.reshape(-1, 1))
    return train_df, scaler_age

def weight_scaling_log(train_df):
    train_df['Weight'] = train_df['Weight'].astype(float)
    train_df['Log_Weight'] = np.log1p(train_df['Weight'])
    return train_df

def weight_scaling_minmax(train_df):
    train_df['Weight'] = train_df['Weight'].astype(float)
    scaler_weight = MinMaxScaler()
    train_df['Scaled_Weight'] = scaler_weight.fit_transform(train_df['Weight'].values.reshape(-1, 1))
    return train_df, scaler_weight

def height_scaling_log(train_df):
    train_df['Log_Height'] = np.log1p(train_df['Height'])
    return train_df

def height_scaling_minmax(train_df):
    scaler_height = MinMaxScaler()
    train_df['Scaled_Height'] = scaler_height.fit_transform(train_df['Height'].values.reshape(-1, 1))
    return train_df, scaler_height

def make_gender_binary(train):
    train['Gender'] = train['Gender'].map({'Female':1, 'Male':0})
    return train

def fix_binary_columns(train):
    Binary_Cols = ['family_history_with_overweight','FAVC', 'SCC','SMOKE']
    # if yes then 1 else 0
    for col in Binary_Cols:
        train[col] = train[col].map({'yes': 1, 'no': 0})
    return train

def freq_cat_cols(train):
    # One hot encoding
    cat_cols = ['CAEC', 'CALC']
    for col in cat_cols:
        train[col] = train[col].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})
    return train

def Mtrans(train):
    """
    Public_Transportation    8692
    Automobile               1835
    Walking                   231
    Motorbike                  19
    Bike                       16
    """
    # train['MTRANS'] = train['MTRANS'].map({'Public_Transportation': 3, 'Automobile': 5, 'Walking': 1, 'Motorbike': 4, 'Bike': 2})
    # dummify column
    train = pd.get_dummies(train, columns=['MTRANS'])
    return train


def other_features(train):
    train['BMI'] = train['Weight'] / (train['Height'] ** 2)
    # train['Age'*'Gender'] = train['Age'] * train['Gender']
    polynomial_features = PolynomialFeatures(degree=2)
    X_poly = polynomial_features.fit_transform(train[['Age', 'BMI']])
    poly_features_df = pd.DataFrame(X_poly, columns=['Age^2', 'Age^3', 'BMI^2', 'Age * BMI', 'Age * BMI^2', 'Age^2 * BMI^2'])
    train = pd.concat([train, poly_features_df], axis=1)
    return train


def test_pipeline(test, scaler_age, scaler_weight, scaler_height):
    test = datatypes(test)
    test = encode_target(test)
    test = age_binning(test)
    test = age_scaling_log(test)
    test['Scaled_Age'] = scaler_age.transform(test['Age'].values.reshape(-1, 1))
    test = weight_scaling_log(test)
    test['Scaled_Weight'] = scaler_weight.transform(test['Weight'].values.reshape(-1, 1))
    test = height_scaling_log(test)
    test['Scaled_Height'] = scaler_height.transform(test['Height'].values.reshape(-1, 1))
    test = make_gender_binary(test)
    test = fix_binary_columns(test)
    test = freq_cat_cols(test)
    test = Mtrans(test)
    test = other_features(test)

    return test

def train_model(params, X_train, y_train):
    lgb_train = lgb.Dataset(X_train, y_train)
    model = lgb.train(params, lgb_train, num_boost_round=1000)
    return model

def evaluate_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    y_pred = [np.argmax(y) for y in y_pred]
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

def objective(trial, X_train, y_train):
    params = {
        'objective': 'multiclass',
        'num_class': 7,
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 0.95),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.95),
        'verbosity': -1
    }

    n_splits = 5
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in kf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

        model = train_model(params, X_tr, y_tr)
        accuracy = evaluate_model(model, X_val, y_val)
        scores.append(accuracy)

    return np.mean(scores)

def optimize_hyperparameters(X_train, y_train, n_trials=2):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=n_trials)
    return study.best_params

def New_Test_Instances_Pipeline(test, scaler_age, scaler_weight, scaler_height):
    test = datatypes(test)
    test = age_binning(test)
    test = age_scaling_log(test)
    test['Scaled_Age'] = scaler_age.transform(test['Age'].values.reshape(-1, 1))
    test = weight_scaling_log(test)
    test['Scaled_Weight'] = scaler_weight.transform(test['Weight'].values.reshape(-1, 1))
    test = height_scaling_log(test)
    test['Scaled_Height'] = scaler_height.transform(test['Height'].values.reshape(-1, 1))
    test = make_gender_binary(test)
    test = fix_binary_columns(test)
    test = freq_cat_cols(test)
    test = Mtrans(test)
    test = other_features(test)

    return test

Experiment

In [7]:
path = '/Users/arham/Downloads/Projects/01-Dataset/01-Data-for-model-building/train.csv'
train_df, val_df, test_df = load_data(path)

# train test val pipeline
train_df = datatypes(train_df)
train_df = encode_target(train_df)
train_df = age_binning(train_df)
train_df, scaler_age = age_scaling_minmax(train_df)
train_df = age_scaling_log(train_df)
train_df, scaler_weight = weight_scaling_minmax(train_df)
train_df = weight_scaling_log(train_df)
train_df, scaler_height = height_scaling_minmax(train_df)
train_df = height_scaling_log(train_df)
train_df = make_gender_binary(train_df)
train_df = fix_binary_columns(train_df)
train_df = freq_cat_cols(train_df)
train_df = Mtrans(train_df)
train_df = other_features(train_df)
val_df = test_pipeline(val_df, scaler_age, scaler_weight, scaler_height)
test_df = test_pipeline(test_df, scaler_age, scaler_weight, scaler_height)


# target & predictors
Target = 'NObeyesdad'
features = ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'Age_Group', 
       'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking', 'BMI', 'Age^2',
       'Age^3', 'BMI^2', 'Age * BMI', 'Age * BMI^2', 'Age^2 * BMI^2'] 
        #'Scaled_Age', 'Log_Age', 'Scaled_Weight', 'Log_Weight', 'Scaled_Height', 'Log_Height',

X_train = train_df[features]
y_train = train_df[Target]
X_val = val_df[features]
y_val = val_df[Target]
X_test = test_df[features]
y_test = test_df[Target]


lgb_train = lgb.Dataset(X_train, y_train)
params = {
    'objective': 'multiclass',
    'num_class': 7,
    'metric': 'multi_logloss',
}

from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import mlflow
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_predict

mlflow.sklearn.autolog(disable=True)

with mlflow.start_run(run_name="LGBM_without_FE_v2"):
    class_counts_train = [y_train[y_train == i].count() / y_train.count() for i in range(7)]
    class_counts_val = [y_val[y_val == i].count() / y_val.count() for i in range(7)]
    target_drift = [(train_count - val_count) for train_count, val_count in zip(class_counts_train, class_counts_val)]
    print(f"Target Drift For Each Class {target_drift}")
    mlflow.log_params({'Target_Drift_' + str(i): freq for i, freq in enumerate(target_drift)})

    model = LGBMClassifier(**params)  # Assuming you have your parameters defined somewhere
    model.fit(X_train, y_train)  # Fit the model on training data

    # CV predictions of LightGBM
    cv_predictions = cross_val_predict(model, X_train, y_train, cv=5)
    accuracy_lgbm = accuracy_score(y_train, cv_predictions)
    
    # Compute precision, recall, and F1-score
    precision_lgbm, recall_lgbm, f1_lgbm, _ = precision_recall_fscore_support(y_train, cv_predictions, average='weighted')
    
    print("\nAccuracy:", accuracy_lgbm)
    print("Precision:", precision_lgbm)
    print("Recall:", recall_lgbm)
    print("F1:", f1_lgbm)
    
    mlflow.log_metric('accuracy', accuracy_lgbm)
    mlflow.log_metric('precision', precision_lgbm)
    mlflow.log_metric('recall', recall_lgbm)
    mlflow.log_metric('f1', f1_lgbm)

    model.fit(X_train, y_train)
    y_val_pred_lgbm = model.predict(X_val)
    
    # Compute precision, recall, and F1-score per class
    precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(y_val, y_val_pred_lgbm, average=None)
    for i in range(len(recall_per_class)):
        print(f"Recall for class {i}: {recall_per_class[i]}")
        mlflow.log_metric(f'recall_class_{i}', recall_per_class[i])

    mlflow.lightgbm.log_model(model, 'model')
    mlflow.set_tag('experiments', 'Arham A.')
    mlflow.set_tag('model_name', 'LGBM')
    mlflow.set_tag('preprocessing', 'Yes')


Target Drift For Each Class [0.004943133623686147, 0.011990707821925795, -0.0087675011457998, -0.001077949504617301, -0.017190035106736085, -0.00032756263090533144, 0.01042920694244659]
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001163 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3576
[LightGBM] [Info] Number of data points in the train set: 10793, number of used features: 25
[LightGBM] [Info] Start training from score -2.103541
[LightGBM] [Info] Start training from score -1.893390
[LightGBM] [Info] Start training from score -2.159762
[LightGBM] [Info] Start training from score -2.113461
[LightGBM] [Info] Start training from score -1.974767
[LightGBM] [Info] Start training from score -1.867272
[LightGBM] [Info] Start training from score -1.619963
[Li

In [39]:
# train_df, val_df, test_df = load_data(path)


# X_val = val_df[features]
# y_val = val_df[Target]
# y_pred = model.predict(X_val, num_iteration=model.best_iteration)
# # y_pred to a dataframe
# y_pred = pd.DataFrame(y_pred, columns=['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II', 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III'])
# # add prefix to columns "prob_lgbm_"
# y_pred = y_pred.add_prefix('prob_lgbm_')
# # add to X_val
# X_val = pd.concat([X_val, y_pred], axis=1)
# # export as stack_aid_lgbm.csv
# X_val.to_csv('stack_aid_lgbm.csv', index=False)

In [25]:
# show all columns
pd.set_option('display.max_columns', None)
X_train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,Age_Group,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,BMI,Age^2,Age^3,BMI^2,Age * BMI,Age * BMI^2,Age^2 * BMI^2
0,1,21.000000,1.550000,51.000000,0,1,3.0,1.0,2,0,2.000000,0,0.000000,0.000000,0,21-30,False,False,False,True,False,21.227888,1.0,21.000000,21.227888,441.000000,445.785640,450.623213
1,0,20.000000,1.700000,80.000000,1,1,2.0,3.0,1,0,2.000000,0,2.000000,1.000000,1,0-20,False,False,False,True,False,27.681661,1.0,20.000000,27.681661,400.000000,553.633218,766.274350
2,1,18.000000,1.600000,60.000000,1,1,2.0,3.0,1,0,2.000000,0,1.000000,1.000000,0,0-20,False,False,False,False,True,23.437500,1.0,18.000000,23.437500,324.000000,421.875000,549.316406
3,1,26.000000,1.632983,111.720238,1,1,3.0,3.0,1,0,2.559750,0,0.000000,0.396972,1,21-30,False,False,False,True,False,41.895611,1.0,26.000000,41.895611,676.000000,1089.285877,1755.242193
4,1,21.682636,1.748524,133.845064,1,1,3.0,3.0,1,0,2.843777,0,1.427037,0.849236,1,21-30,False,False,False,True,False,43.778327,1.0,21.682636,43.778327,470.136704,949.229536,1916.541944


Testing Single Instance For Architecture Development

In [37]:
input_data = {
    "id": 6204,
    "Gender": "Female",
    "Age": 23.0,
    "Height": 1.581527,
    "Weight": 78.089575,
    "family_history_with_overweight": "yes",
    "FAVC": "yes",
    "FCVC": 2.0,
    "NCP": 2.070033,
    "CAEC": "Sometimes",
    "SMOKE": "no", 
    "CH2O": 2.953192,
    "SCC": "no",
    "FAF": 0.118271,
    "TUE": 0.0,
    "CALC": "no",
    "MTRANS": "Public_Transportation"
    
}

input_df = pd.DataFrame([input_data])
input_df = New_Test_Instances_Pipeline(input_df, scaler_age, scaler_weight, scaler_height)

# X input to have same columns as features
X_input = pd.DataFrame(columns=features)
# if input df does not have a column that is in features, add it with 0s at the same position
for col in features:
    if col not in input_df.columns:
        if col in ['MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking']:
            X_input[col] = False
        else:
            X_input[col] = 0
    else:
        X_input[col] = input_df[col]
    # if MTRANS_Automobile, MTRANS_Bike, MTRANS_Motorbike, MTRANS_Public_Transportation, MTRANS_Walking are zero, make them False
    
y_pred_proba = model.predict(X_input)
y_pred = np.argmax(y_pred_proba)

y_pred

4